#  <center>实验
    
## 线性规划方法

- 模型产生数据

<center>$y=1+2x+e,e \in N(0,1)$</center>
    
  $n=30，p=2 $
  
-  代码

In [ ]:
'''
import numpy as np
import pandas as pd
import math
import scipy
from scipy.optimize import minimize
from scipy.optimize import linprog
n=30
tau=0.5
p=2
x = np.random.normal(0 ,1,n)
y = 1+2*x+ np.random.normal(0 ,1,n)
'''
linear programing
使用线性规划包linpro，内点法
运行函数前先把必要的库运行好
'''
def fit1(tau):
    from scipy.optimize import minimize
    from scipy.optimize import linprog
    A=np.hstack([np.zeros(p),tau*np.ones(n),(1-tau)*np.ones(n)])
    a=np.ones(n).T
    b=np.array(x)
    B=np.vstack([a,b,np.eye(n),-np.eye(n)])
    B=B.T
    res = linprog(A, A_eq=B, b_eq=y)

    beta_solve1=res.x[:p]#线性规划的解

    u=res.x[p+1:p+1+n]
    v=res.x[p+2+n:]
    #u-v#线性规划所得的残差
    return beta_solve1
'''

- outcome: $[0.85542775, 2.27101859]$

## 次梯度方法(SubgradientMethod)

- 模型产生数据

和上文一样，只是令 $n=300$

- 代码

In [ ]:
'''
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
import math
import scipy
import time    
n=300#最开始以n=30实验
tau=0.5
p=2
x = np.random.normal(10,1,n)
y = 1+2*x+ np.random.normal(0,1,n)

X=np.vstack((np.ones(n),x))
X=X.T
beta_new=np.ones(p)
beta_old=np.zeros(p)
n_iter = 0
start = time.clock()

diff = np.max(np.abs(beta_new - beta_old))

history = dict(params = [], mse=[])
while n_iter<1000 and diff > 0.001:
    beta_old=beta_new
    predict=np.dot(X,beta_old)
    e=y-predict
    
    mask = e==0
    sub1=np.where(e>0,tau*X[:,1],(tau-1)*X[:,1])
    sub1[mask]=random.uniform((tau-1)*X[:,1][mask],tau*X[:,1][mask])

    sub0=np.where(e>0,tau*X[:,0],(tau-1)*X[:,0])
    sub0[mask]=random.uniform((tau-1)*X[:,0][mask],tau*X[:,0][mask])


    sub=np.vstack((sub0,sub1))
    sub_gradient=np.array([np.mean(sub[0]),np.mean(sub[1])])
    beta_new=beta_old+0.01*sub_gradient
    diff = np.max(np.abs(beta_new - beta_old))
    resid=np.abs(e)
    history['params'].append(diff)
    history['mse'].append(np.mean(resid*resid))
    print(n_iter)
    n_iter += 1


elapsed = (time.clock() - start)
print(elapsed)


#残差图
plt.scatter(x,y)
plt.plot(x,beta_new[0]+beta_new[1]*x)
plt.show()
#待估参数收敛速度
plt.plot(history["params"])
'''

- **$outcome$**:

 运行时间：0.011480500000061511
 
 迭代次数：27

* **<center> 次梯度算法拟合效果 </center>**
    
 ![image1.png](sub_gradient.png)

- **<center>待估参数收敛速度</center>**

 ![image1.png](sub_gradient1.png)

## Bernstein Polynomials Method

$\rho_\tau(e) is continous \quad e \in[a \cdot b]$ 可以推导出在$[a \cdot b]$的Bernstein Polynomials

$\sum_{i=0}^{n} \rho_\tau\left(\frac{(n-i) a+i b}{n}\right) \cdot C_{n}^{i}\left(\frac{e-a}{b-a}\right)^{i}\left(\frac{b-e}{b-a}\right)^{n-i}$

为检验拟合效果，生成数据501个，服从$[-3 \cdot 3]$的均匀分布多项式展开水平为20，拟合分位损失函数($\tau$ 分别取$0,0.1,0.3,0.7,0.9,1$)的图

- **代码**


In [ ]:
'''
def bernstein_poly_ab ( n, a, b, x ): 
    import numpy as np
    from sys import exit
    p = np.zeros(n + 1)      
    p[0] = ( b - x ) / ( b - a )
    p[1] = ( x - a ) / ( b - a ) 
    for i in range ( 2, n + 1 ):
        p[i] = ( x - a ) * p[i-1] / ( b - a )
        for j in range ( i - 1, 0, -1 ):
            p[j] = ( ( b - x ) * p[j] + ( x - a ) * p[j-1] ) / ( b - a )
        p[0] = ( b - x ) * p[0] / ( b - a )
    return p

------------------------------------------------------------------------
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
import math
import scipy
import platform
from scipy.optimize import curve_fit
import time
from sys import exit

a = -3.0
b = 3.0
#####################
for degree in range ( 0, 21 ):
    xdata = np.zeros ( degree + 1 )
    ydata = np.zeros ( degree + 1 )
    for i in range ( 0, degree + 1 ):
        if ( degree == 0 ):
            xdata[i] = 0.5 * ( a + b );
        else:
            xdata[i] = ( float ( degree - i ) * a   \
             + float (          i ) * b ) \
             / float ( degree     )

        ydata[i] = np.sin ( xdata[i] )
############################
degree=20       
xdata=np.linspace(a,b,degree+1)
tau=1.0
ydata=np.where(xdata>0,tau*xdata,(tau-1)*xdata)

nval = 501

xval = np.linspace ( a, b, nval )

from bernstein_poly_ab import bernstein_poly_ab
yval = np.zeros ( nval )
for i in range ( 0, nval ):
   bvec = bernstein_poly_ab ( degree, a, b, xval[i] )
   yval[i] = np.dot ( ydata, bvec )
   
   
   
plt.plot(xval,yval)
plt.plot(xdata,ydata)
plt.show()


error =   abs ( yval - np.where(xval>0,tau*xval,(tau-1)*xval) ) 

error_max = max(error)

- 

![image3.png](tau(0.0).png)

![image4.png](tau(0.1).png)

![image5.png](tau(0.3).png)

![image.png](tau(0.5).png)

![image6.png](tau(0.7).png)

![image7.png](tau(0.9).png)

![image8.png](tau(1.0).png)

Quantile loss function:

$\rho_\tau(Y-X \beta) \quad Y-X \beta \in[a \cdot b]$

the Bernstein Polynomials of quantile loss (Let $e=Y-X\beta$):

$\sum_{i=0}^{n} \rho_\tau\left(\frac{(n-i) a+i b}{n}\right) \cdot C_{n}^{i}\left(\frac{(Y-X \beta)-a}{b-a}\right)^{i}\left(\frac{b-(Y-X \beta)}{b-a}\right)^{n-i}$

但是如果要是估计$\beta$的话，用高斯牛顿迭代的话，每迭代一次，$e$的区间$[a \cdot b]$会发生变化，

一阶导数为：

$\sum_{i=1}^{n-1} \rho_\tau\left(\frac{(n-i) a+i b}{n}\right) \cdot C_{n}^{i}\left(\frac{(Y-X \beta)-a}{b-a}\right)^{i-1}\left(\frac{b-(Y-X \beta)}{b-a}\right)^{n-i-1}\left(n-2i\right)X+ \rho_\tau(a) \left(\frac{b-(Y-X \beta)}{b-a}\right)^{n-1} X +\rho_\tau(a) \left(\frac{(Y-X \beta)-a}{b-a}\right)^{i-1} \left( -X \right)$

是否每迭代一次，$[a \cdot b]$都需要更新一次